In [23]:
# data.py

from os.path import join

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import pytorch_lightning as pl

import pandas as pd


def lymph_collate_fn(x):
    image, tabular = x
    

class lymph_dataset(Dataset):
    """Wrapper around an imagefolder dataset to provide with tabular data"""
    def __init__(self, path, csv_path, transforms):
        super().__init__()
        self.path = path
        self.csv_path = csv_path
        self.transforms = transforms
        self.images = torchvision.datasets.ImageFolder(root=self.path, transform=self.transforms)
        self.tabular = pd.read_csv(self.csv_path, index_col=0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image, lab_idx = self.images[idx]
        label = self.images.classes[lab_idx]
        tab_data = self.tabular.loc[label]
        return image, tab_data.LABEL


class lymph_datamodule(pl.LightningDataModule):
    def __init__(self, path, batch_size=32, train_prop=0.8):
        super().__init__()
        self.train_path = join(path, "trainset")
        self.train_csv_path = join(self.train_path, "trainset_true.csv")
        self.test_path = join(path, "testset")
        self.test_csv_path = join(self.test_path, "testset_data.csv")
        self.batch_size = batch_size
        self.train_prop = train_prop

        # TODO data augment
        # TODO normalize?
        self.train_transforms = transforms.Compose([
            torchvision.transforms.Resize(224),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor()
        ])
        self.test_transforms = transforms.Compose([
            torchvision.transforms.Resize(224),
            torchvision.transforms.ToTensor()
        ])

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.full_dataset = lymph_dataset(self.train_path, self.train_csv_path, self.train_transforms)
            train_size = int(len(self.full_dataset) * self.train_prop)
            val_size = len(self.full_dataset) - train_size
            self.train_dataset, self.val_dataset = random_split(self.full_dataset, [train_size, val_size])

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.test_dataset = lymph_dataset(self.test_path, self.test_csv_path, self.test_transform)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset,
                                           batch_size=self.batch_size,
                                           shuffle=True,
                                           num_workers=8,
                                           pin_memory=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,
                                           batch_size=self.batch_size,
                                           shuffle=True,
                                           num_workers=8,
                                           pin_memory=True)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset,
                                           batch_size=self.batch_size,
                                           shuffle=True,
                                           num_workers=8,
                                           pin_memory=True)

In [38]:
# model.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.optim import SGD, Adam
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import pytorch_lightning as pl


# Some code reused from https://github.com/Stevellen/ResNet-Lightning/blob/master/resnet_classifier.py
class TransferResNet(pl.LightningModule):
    def __init__(self,
                 resnet_version=18,
                 optimizer='adam',
                 lr=1e-3):
        super().__init__()
        self.__dict__.update(locals())

        resnets = {
            18: models.resnet18, 34: models.resnet34,
            50: models.resnet50, 101: models.resnet101,
            152: models.resnet152
        }
        optimizers = {'adam': Adam, 'sgd': SGD}

        self.optimizer = optimizers[optimizer]
        
        self.criterion = nn.BCEWithLogitsLoss()

        self.resnet = resnets[resnet_version](pretrained=True)
        linear_size = list(self.resnet.children())[-1].in_features
        self.resnet.fc = nn.Linear(linear_size, 1)

        for child in list(self.resnet.children())[:-1]:
            for param in child.parameters():
                param.requires_grad = False
        self.resnet.eval()

    def forward(self, im):
        return self.resnet(im).squeeze()

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)
    
    def training_step(self, batch, batch_idx):
        image, label = batch
        score = self(image)
        loss = self.criterion(score, label.float())
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        image, label = batch
        score = self(image)
        loss = self.criterion(score, label.float())
        pred = (score > 0.5).int()
        acc = (pred == label).sum() / pred.shape[0]
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss


In [39]:
# train.py

from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint


pl.seed_everything(42)
    
# Handle the data
dm = lymph_datamodule("data", batch_size=16)

# Define model
model = TransferResNet()

# Exp logger
logger = TensorBoardLogger('logs/tensorboard_logs')

# Define training
checkpointer = ModelCheckpoint(monitor='val_loss',
                               save_top_k=3,
                               mode='min',
                               save_last=True,
                               filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}')
trainer = pl.Trainer(gpus=1,
                     max_epochs=10,
                     #callbacks=[checkpointer],
                     logger=logger,
                     val_check_interval=0.5)

# Train
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | criterion | BCEWithLogitsLoss | 0     
1 | resnet    | ResNet            | 11.2 M
------------------------------------------------
513       Trainable params
11.2 M    Non-trainable params
11.2 M    Total params


Epoch 0:  33%|███▎      | 336/1011 [01:16<02:33,  4.39it/s, loss=0.352, v_num=20]
Validating: 0it [00:00, ?it/s]
Validating:  78%|███████▊  | 132/169 [00:31<00:12,  3.06it/s]


Epoch 0:  83%|████████▎ | 842/1011 [03:45<00:45,  3.73it/s, loss=0.374, v_num=20]
Validating: 0it [00:00, ?it/s]
Validating:  56%|█████▌    | 94/169 [00:23<00:20,  3.70it/s]


Epoch 1:  33%|███▎      | 336/1011 [01:42<03:25,  3.28it/s, loss=0.419, v_num=20] 
Validating: 0it [00:00, ?it/s]
Validating:  33%|███▎      | 56/169 [00:11<00:28,  3.92it/s]


Epoch 1:  83%|████████▎ | 842/1011 [04:05<00:49,  3.43it/s, loss=0.352, v_num=20]
Validating: 0it [00:00, ?it/s]
Validating:  11%|█         | 18/169 [00:02<00:35,  4.30it/s]


Epoch 1:  96%|█████████▌| 972/1011 [04:40<00:11,  3.46it/s, loss=0.352, v_num=20]


1

In [ ]:
dm = lymph_datamodule("data", batch_size=16)
dm.setup("fit")
print(type(dm.train_dataset.dataset.tabular.loc["P0"].values))

In [ ]:
torchvision.datasets.ImageFolder("data/trainset", transform=torchvision.transforms.ToTensor())